In [ ]:
#balance_data
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load('training_data.npy')

df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))

lefts = []
rights = []
forwards = []

shuffle(train_data)

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1,0,0]:
        lefts.append([img,choice])
    elif choice == [0,1,0]:
        forwards.append([img,choice])
    elif choice == [0,0,1]:
        rights.append([img,choice])
    else:
        print('no matches')


forwards = forwards[:len(lefts)][:len(rights)]
lefts = lefts[:len(forwards)]
rights = rights[:len(forwards)]
final_data = forwards + lefts + rights
shuffle(final_data)

np.save('training_data_v2.npy', final_data)



In [2]:
import torch
import torch.nn as nn
import numpy as np

class Flatten(nn.Module):
    def forward(self,input):
        return input.view(input.shape[0],-1)
        

model = nn.Sequential(
            nn.Conv2d(1,64,3,4,2), #
            nn.ReLU(),
            nn.Conv2d(64,192,3,2,2),
            nn.ReLU(),
            nn.Conv2d(192,256,3,2,2),
            nn.ReLU(),
            #features
            Flatten(),
            #classifier
            nn.Linear(10752,2048),
            nn.ReLU(),
            nn.Linear(2048,1024),
            nn.ReLU(),
            nn.Linear(1024,3)
            )
WIDTH = 80
HEIGHT = 60 
lr = 1e-3
EPOCHS = 8
train_data = np.load('training_data_v2.npy')

train = train_data[:-500]
test = train_data[-500:]

X = torch.tensor(np.array([i[0] for i in train]).reshape(-1,1,WIDTH,HEIGHT)).float()
Y = torch.tensor([i[1] for i in train]).float()
test_x = torch.tensor(np.array([i[0] for i in test]).reshape(-1,1,WIDTH,HEIGHT)).float()
test_y = torch.tensor([i[1] for i in test]).float()


criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr)


#train
for epoch in range(EPOCHS):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()






In [7]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(test_x)
    total += test_y.size(0)
    correct += (np.around(outputs) == test_y).sum().item()
print(np.around(outputs),test_y)
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [-0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]) tensor([[ 0.,  0.,  1.],
        [ 0.,  0.,  1.],
        [ 1.,  0.,  0.],
        ...,
        [ 0.,  1.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.]])
Test Accuracy of the model on the 10000 test images: 200.0 %


In [8]:
import numpy as np
import cv2
import time
import pyautogui
from directkeys import PressKey,ReleaseKey, W, A, S, D
from draw_lanes import draw_lanes
from grabscreen import grab_screen
from getkeys import key_check
from create_training_data import keys_to_output
import os
from PIL import ImageGrab

File exists, loading previous data!


In [15]:
def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

def left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)

def right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    
def main():

    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)
    last_time = time.time()
    paused = False
    while(True):
        
        if not paused:
            # 800x600 windowed mode
            screen =  np.array(ImageGrab.grab(bbox=(0,40,800,640)))
            print('loop took {} seconds'.format(time.time()-last_time))
            last_time = time.time()
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
            screen = cv2.resize(screen, (80,60))
            moves = np.around(model(torch.tensor(screen.reshape(1,1,80,60),
                                                 requires_grad=False).float()))
            if moves == [1,0,0]:
                left()
            elif moves == [0,1,0]:
                straight()
            elif moves == [0,0,1]:
                right()
            else :
                straight()
   
        keys = key_check()

        # p pauses game and can get annoying.
        if 'T' in keys:
            if paused:
                paused = False
                time.sleep(1)
            else:
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)
    
if __name__=='__main__':
    main()

4
3
2
1
loop took 0.051886796951293945 seconds


RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.